Now that we have our data, let's start cleaning and formatting. This notebook focuses on the Python Pandas library.<br>
<br>N.B. When viewing the columns, note the following prefixes:<br>
_v :	Calculated value (max, min or mean)<br>
_s :	Standard deviation of mean<br>
_c :	Count of (number of) values included<br>
_d :	Date range for values<br>
_y :	Years where extreme occurred (limited to first 40) <br><br>
(https://www.weatherstats.ca/faq/#download-columns-ne)

In [ ]:
""" Load the raw data """

# import
import pandas as pd

df = pd.read_csv("/dbfs/mnt/weather/raw/weather_data.csv")
display(df)

In [ ]:
""" Overview of the data """

# view the fields
print(df.columns)

# view the row count and column count
print(df.shape)

# # view first 5 rows
display(df.head())

# # view last 5 rows
display(df.tail())

# view summary stats
# display(df.info())
display(df.describe())  # other way to view stats

In [ ]:
# more advanced summary stats, unique to Databricks
dbutils.data.summarize(df)

In [ ]:
""" Select relevant columns and fix the data types """

# verify data types
display(df.dtypes)


# select desired columns, rename them, and fix the data types
df_clean = df.rename(
    columns={
        "max_dew_point_v": "max_dew_point",
        "max_relative_humidity_v": "max_relative_humidity",
        "max_temperature_v": "max_temperature",
        "max_wind_speed_v": "max_wind_speed",
        "min_dew_point_v": "min_dew_point",
        "min_relative_humidity_v": "min_relative_humidity",
        "min_temperature_v": "min_temperature",
        "min_wind_speed_v": "min_wind_speed",
        "precipitation_v": "precipitation",
        "snow_v": "snow",
        "snow_on_ground_v": "snow_on_ground",
    }
)[
    [
        "date",
        "max_dew_point",
        "max_relative_humidity",
        "max_temperature",
        "max_wind_speed",
        "min_dew_point",
        "min_relative_humidity",
        "min_temperature",
        "min_wind_speed",
        "precipitation",
        "snow",
        "snow_on_ground",
    ]
].astype(
    {
        "date": "datetime64",
        "max_dew_point": "float",
        "max_relative_humidity": "float",
        "max_temperature": "float",
        "max_wind_speed": "float",
        "min_dew_point": "float",
        "min_relative_humidity": "float",
        "min_temperature": "float",
        "min_wind_speed": "float",
        "precipitation": "float",
        "snow": "float",
        "snow_on_ground": "float",
    }
)
df_clean["date"] = pd.to_datetime(df_clean["date"]).dt.date
display(df_clean)

In [ ]:
""" Handle missing values """

# import
from datetime import date

# find the number of null rows
print(df_clean.isna().sum())

# exclude the 'date' column and find the number of rows that have null in every column
df_subset = df_clean.drop("date", axis=1)
print(df_subset.isnull().all(axis=1).sum())

# there are 360 rows with nulls in every column, let's now see what they are
display(df_clean[df_subset.isnull().all(axis=1)])

# we see that the data is null starting from 1872-06-23 and anything earlier.
df_clean = df_clean[
    df_clean["date"] > date(1872, 6, 23)
]  # drop the rows that have a 'date' before 1872-06-23
# print(df_clean.isna().sum())

# We can now safely replace the remaining nulls with 0.
df_clean.fillna(0, inplace=True)

# verify that there are no null values left
print(df_clean.isna().sum())

# view the row count and column count
df_clean.shape

In [ ]:
""" Handle duplicated rows """

# verify if there are any duplicates in our current dataframe
print("Duplicated rows of the whole dataframe: ", df_clean.duplicated().sum())

# No duplicates, however let's create a subset of the data that excludes the 'date' column
df_subset = df_clean.drop("date", axis=1)

# within that subset, find the amount of rows that are duplicated
print("Duplicated rows of the subset: ", df_subset.duplicated().sum())

# now we see that there are 32 rows that are duplicates, let's see what they are
df3 = df_clean[df_subset.duplicated(keep=False)]
# display(df3)

# the duplicated rows are not shown together so let's sort the result
display(df3.sort_values(by="date"))

# we see that the duplicates are mostly focused on very early dates and often are due to a mismatched digit. It is safe to drop them.
df_clean = df_clean[~df_subset.duplicated()]


# view the row count and column count
df_clean.shape

In [ ]:
""" Export the processed dataframe to storage """

# write the result file to processed folder, save as parquet file
df_clean.to_parquet(
    "/dbfs/mnt/weather/processed/weather_processed.parquet", index=False
)